In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import fsolve
from matplotlib.widgets import Button, Slider
from matplotlib.widgets import Slider
from super_cub import *

In [3]:


params = AircraftParams(
    bw=2.28,
    cw=0.25,
    a2dw=0.1,
    alpha_0w=-4,
    ew=0.9,
    cd0w=0.007,
    rw=0.002,
    Cmw=-0.1,
    bt=0.5,
    ct=0.1,
    cef=0.4*0.1,
    lt=2.5,
    tt=0.00635,
    ih=0.,
    et=0.99,
    a2dt=0.1,
    alpha_0t=-4,
    epsilon0=0,
    tau_e=0.7,
    cd0t=0.007,
    rt=0.002,
    df=0.2,
    lf=2,
    htv=0.2,
    ltv=0.8
)



In [5]:
airplane = super_cub(params)

AttributeError: 'super_cub' object has no attribute 'et'

In [ ]:
def generate_data():
    alphas = np.linspace(-6.27, 15, 100)
    del_e = np.linspace(-10, 10, 5)
    Re_c = 500000
    airplane = super_cub()
    h = 0.4
    Cms = np.zeros((len(del_e), len(alphas)))
    Cls = np.zeros((len(del_e), len(alphas)))
    Cds = np.zeros((len(del_e), len(alphas)))

    for i in range(len(del_e)):
        for j in range(len(alphas)):
            Cms[i,j] = airplane.get_coefficients(alphas[j], del_e[i], Re_c, h)[0]
            Cls[i,j] = airplane.get_coefficients(alphas[j], del_e[i], Re_c, h)[1]
            Cds[i,j] = airplane.get_coefficients(alphas[j], del_e[i], Re_c, h)[2]
        


    # Define the del_e values
    del_e_values = np.linspace(-10, 10, 5)
    alpha_trim = np.linspace(-18,18, 100)
    trim_cl = []
    trim_cd = []

    for alpha in alpha_trim:
        f_CM = lambda x: airplane.get_coefficients(5, x, Re_c, h)[0]
        del_e = fsolve(f_CM, 0)[0]
        trim_cl.append(airplane.get_coefficients(alpha, del_e, Re_c, h)[1])
        trim_cd.append(airplane.get_coefficients(alpha, del_e, Re_c, h)[2])

    from scipy.optimize import curve_fit

    def parabolic_function(x, cd0, K):
        return cd0 + K*x**2

    popt, _ = curve_fit(parabolic_function, trim_cl, trim_cd)
    coefficients = np.polyfit(trim_cl, trim_cd, 2)

    # Extract the coefficients
    cd0 = popt[0]
    K = popt[1]

 

    return Cls, Cds, Cms, trim_cd, trim_cl, cd0, K